In [ ]:
import numpy as np
import pandas as pd
import os
import time
import schedule
import joblib
import datetime
import multiprocessing as mp
from get_strategy_meta import get_all_strategy_meta
from query_api import get_path_dict, gen_connection, load_item_by_key, query_mysql
from get_signal_mongo import write_signal_mongo, get_signal_mongo
from get_signal_mysql import write_signal, get_signal
from mongo_functions import get_portfolio_db

In [ ]:

db_name = "MLProduction"
coll_name = "SignalOneBB2"
db = get_portfolio_db(db_name)
coll = db[coll_name]
# r = coll.find({"code":"Z_C007", "key":"s_AUDUSD"})
r = coll.find()
df = pd.DataFrame([j for j in r])
# df.sort_values("time")
df.tail()

In [ ]:
coll.find({"code": "W006"}).distinct("key")

In [51]:
# Check Log
db_name = "MLLog"
coll_name = "log"
db = get_portfolio_db(db_name)
coll = db[coll_name]
# r = coll.find({"code":"W006", "key":"s_AUDUSD"})
# r = coll.find({"code": "X004", "key": "s_TLT"})
r = coll.find({"idd.code": "Y001"})
df = pd.DataFrame([j for j in r])
# df.sort_values("time")000.6 
df.tail()

,Date,_id,idd,message,other
4723,2019-07-09 21:47:46.034,5d2543c28b634c5bf502db37,"{'code': 'Y001', 'key': 's_XLE'}",Scaling: Done!,
4724,2019-07-09 21:47:46.035,5d2543c28b634c5bf502db38,"{'code': 'Y001', 'key': 's_XLE'}",TimeseriesGenerator: Done!,
4725,2019-07-09 21:47:50.375,5d2543c68b634c5bf502db39,"{'code': 'Y001', 'key': 's_XLE'}",Model Loaded!,
4726,2019-07-09 21:47:51.182,5d2543c78b634c5bf502db3a,"{'code': 'Y001', 'key': 's_XLE'}",Predict: Done!,
4727,2019-07-09 21:47:51.183,5d2543c78b634c5bf502db3b,"{'code': 'Y001', 'key': 's_XLE'}",s_XLE,"{'last_date': 2019-07-09 16:00:00, 'query_date..."


In [52]:
df.head()

,Date,_id,idd,message,other
0,2019-07-08 04:32:11.723,5d22ff8b545037c58f3ae5b2,"{'code': 'Y001', 'key': 'b_EMB'}","Loaded: scaler, DATA_PARAMS, MODEL_PARAMS, model.",
1,2019-07-08 04:32:11.746,5d22ff8b545037c58f3ae5b3,"{'code': 'Y001', 'key': 'b_EMB'}",Loaded: MINUTE_INDEX for EMB,
2,2019-07-08 04:32:11.746,5d22ff8b545037c58f3ae5b4,"{'code': 'Y001', 'key': 'b_EMB'}",Using Mongo: 2019-07-04 04:30:05.703771 - 2019...,
3,2019-07-08 04:32:12.373,5d22ff8c545037c58f3ae5b6,"{'code': 'Y001', 'key': 'b_EMB'}",Scaling: Done!,
4,2019-07-08 04:32:12.375,5d22ff8c545037c58f3ae5b7,"{'code': 'Y001', 'key': 'b_EMB'}",Error at get_signal: `start_index+length=30 > ...,


In [38]:
db_name = "MLProduction"
coll_name = "SignalOneBB3"
db = get_portfolio_db(db_name)
coll = db[coll_name]
# r = coll.find({"code":"W006", "key":"s_AUDUSD"})
# r = coll.find({"code": "X004", "key": "s_TLT"})
r = coll.find({"code": "Y001"})
df = pd.DataFrame([j for j in r])
# df.sort_values("time")000.6 
df.tail()

,_id,code,key,last_date,query_date,status,status_code,timeframe,write_time,y
0,5d25403f17dee7fa00512cff,Y001,b_NZDUSD,2019-07-09 21:30:00,2019-07-09 21:29:56.156,Successful.,1,15,2019-07-09 21:32:47.374,0.819053
1,5d25404da7110a5691512cff,Y001,s_NZDUSD,2019-07-09 21:30:00,2019-07-09 21:29:56.156,Successful.,1,15,2019-07-09 21:33:01.327,0.790568


In [ ]:
coll.distinct("code")

In [ ]:
df.set_index("query_date")["y"].plot()

In [ ]:
# %%time
# Check prices
db_name = "Production"
coll_name = "prices"
db = get_portfolio_db(db_name)
coll = db[coll_name]
name_list = coll.distinct("name")
r = coll.find({"name": "EFA"})
df = pd.DataFrame([j for j in r])
df = df.set_index("time")
df.index.is_monotonic_increasing
df.head()
df.tail()
df["price"].plot()
dp_list = []
for name in name_list:
    r = coll.find({"name": name})
    df = pd.DataFrame([j for j in r])
    df = df.set_index("time")
    print(name, df.index.max()- df.index.min())
    if df.index.duplicated().any():
        dp_list.append(name)
    
print(len(name_list))
",".join(dp_list)

In [37]:
db_name = "Production"
coll_name = "prices"
db = get_portfolio_db(db_name)
coll = db[coll_name]
r = coll.find({"name": "IWM"})
pdf = pd.DataFrame([j for j in r])
pdf
# pdf = pdf.set_index("time")
# d = pdf.index[pdf.index.duplicated()]
# pdf.loc[d]



""


In [ ]:
pdf.index.duplicated()

In [ ]:
df.set_index("query_date")["status_code"].plot()

In [ ]:
# Log Class

        
    
    